<a href="https://colab.research.google.com/github/PawelSmolinski/Psychometria/blob/main/Obliczenia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Przydatne komendy:
ctrl + F9 - uruchom wszytsko
ctrl + /  - komentuj zaznaczone linijki kodu

In [ ]:
# Instalacja pakietów (2-3 minuty)
install.packages("dplyr")
install.packages("psych")
install.packages("lavaan")
install.packages("corrplot")
install.packages("GPArotation")
install.packages("semTools")


library('dplyr')
library('psych')
library('lavaan')
library('corrplot')
library('GPArotation')
library('semTools')

In [40]:
# Wczytanie danych
dane <- read.csv('https://raw.githubusercontent.com/PawelSmolinski/Psychometria/main/Dane/Ciemna%20triada/dane50.csv')

In [ ]:
# Struktura danych
str(dane)

In [42]:
# Prosate zmienianie nazwy
dane <- rename(dane, kraj = country)

# Faktoryzacja zmiennej
dane$kraj <- as.factor(dane$kraj)

In [ ]:
# Wizualizacja macierzy
macierz <- cor(dane[,1:18])

corrplot(macierz, method = 'square')

In [ ]:
# Statystyki opisowe
describe(dane[,1:18])

In [45]:
# Proste rekodowanie
dane$N2 <- 6 - dane$N2

In [ ]:
# Szukanie brakujących wartości
which(is.na(dane), arr.ind = TRUE)

In [47]:
# Rekodowanie wartości brakujących
dane$M3[dane$M3 == 0] <- NA
dane$N6[dane$N6 == 0] <- NA

In [48]:
# Zaawansowane rekodowanie
dane <- dane %>%
  mutate(across(c(N6,N8), ~recode(.,`1` = 5, `2` = 4, `3` = 3, `4` = 2, `5` = 1)))

In [49]:
# Obliczanie wyniku
dane$makiawelizm <- rowMeans(dane[,1:9], na.rm = TRUE)
dane$narcyzm <- rowMeans(dane[,10:18], na.rm = TRUE)

In [ ]:
# Prosty wykres rozkładu
hist(dane$makiawelizm, # Nazwa zmiennej
     breaks = 15, # Liczba przedziałów
     col = "orange", # Kolor wypełnienia
     border = "black", # Kolor obramowania
     main = "Rozkład wyników dla Makiawelizmu", # Tytuł wykresu
     xlab = "Wynik w teście", # Etykieta osi X
     ylab = "Ilość osób z wynikiem") # Etykieta osi Y

In [ ]:
# Test t Studenta
t.test(makiawelizm ~ kraj, dane)

In [ ]:
# Test korelacji
cor.test(dane$makiawelizm, dane$narcyzm)

In [ ]:
# Rzetelność (alfa)
alfa <- alpha(dane[,1:18])
summary(alfa)

In [ ]:
# Eksploracyjna analiza czynnikowa
efa <- fa(dane[,1:18], nfactors =  2)
summary(efa)

In [ ]:
# Wykres osypiska
scree(dane[,1:18])

In [58]:
# Model strukturalny
model <- 'Makiawelizm =~ M1 + M2 + M3 + M4 + M5 + M6 + M7 + M8 + M9
          Narcyzm =~ N1 + N2 + N3 + N4 + N5 + N6 + N7 + N8 + N9'

In [ ]:
# Konfirmacyjna analiza czynnikowa
cfa <- cfa(model, dane)
summary(cfa, fit.measures=TRUE)

In [ ]:
#Rzetelność Omega dla podscal
omegaResults <- semTools::reliability(cfa)
print(omegaResults)